# Face recognition with EfficientNetB2 :
 In this project we have taken a kaggle dataset to classify some human emotions.

The main motive of this project was to use the SOTA model produced by GOOGLE AI **EfficientNet** : *EfficientNetB2*.


The first work is we have docked the notebook with kaggle so that we can directly download the data straight into the collab notebooks directory. Then unzipped the folder and now we can use the data without downloading locally.

In [ ]:
!pip install -q kaggle

!mkdir -p ~/.kaggle
                                  
!cp kaggle.json ~/.kaggle/      #   (!cp [   API token file name   ] ~/.kaggle/)

!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d jonathanoheix/face-expression-recognition-dataset

 80% 97.0M/121M [00:01<00:00, 78.8MB/s]
100% 121M/121M [00:01<00:00, 108MB/s]  


In [ ]:
from zipfile import ZipFile
zf = ZipFile('face-expression-recognition-dataset.zip', 'r')
zf.extractall('/content')
zf.close()

As we know this neural network has a super long neuron layers , we have to use gpu for running it conveniently.
So we are installing tensorflow gpu to run it swiftly.

In [ ]:
!pip install tensorflow-gpu

     |████████████████████████████████| 394.3MB 37kB/s 


Now checking what type of gpu we have been alotted.

In [ ]:
!nvidia-smi

Mon Feb  8 13:36:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Libraries:

In [ ]:
import tensorflow as tf
tf.__version__

'2.4.1'

In [ ]:
from tensorflow.keras.layers import Lambda,Input,Dense,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.efficientnet import EfficientNetB2



In [ ]:
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [ ]:
IMAGE_SIZE=[224,224]
train_path='images/train'

valid_path='images/validation'

Now adding the effnetb2.

In [ ]:
eff=EfficientNetB2(input_shape=IMAGE_SIZE+[3],include_top=False)

31793152/31790344 [==============================] - 1s 0us/step


As we are wanting it to be pretrained we are stopping the layers from training on this data.

In [ ]:
for layer in eff.layers:
  layer.trainable=False

In [ ]:
folders=glob('images/train/*')

In [ ]:
folders

['images/train/angry',
 'images/train/fear',
 'images/train/sad',
 'images/train/disgust',
 'images/train/neutral',
 'images/train/surprise',
 'images/train/happy']

In [ ]:
x=Flatten()(eff.output)

We just want to change the last layer as we want to classify the emotions with desirable classes.

In [ ]:
prediction=Dense(len(folders),activation='softmax')(x)

model=Model(inputs=eff.input,outputs=prediction)

Now checking the model.

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 224, 224, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization[0][0]              
______________________________________________________________________________________________

We can see the model is huge. So we need a lot of time to run it.

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

Now making the datasets for training and testing.

In [ ]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

test_datagen=ImageDataGenerator(rescale=1./255)

In [ ]:
training_set=train_datagen.flow_from_directory(train_path,
                                               target_size=(224,224),batch_size=32,class_mode='categorical')

Found 28821 images belonging to 7 classes.


In [ ]:
test_set=test_datagen.flow_from_directory(valid_path,
                                               target_size=(224,224),batch_size=32,class_mode='categorical')

Found 7066 images belonging to 7 classes.


As running many epoch can take the whole day we are only showing 3 epochs to let you see how the model is getting tuned with the dataset.

In [ ]:
r=model.fit_generator(training_set,validation_data=test_set,
                      epochs=3,
                      steps_per_epoch=len(training_set),
                      validation_steps=len(test_set))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/3
901/901 [==============================] - 2935s 3s/step - loss: 6.9446 - accuracy: 0.1744 - val_loss: 3.5402 - val_accuracy: 0.1612
Epoch 2/3
901/901 [==============================] - 2904s 3s/step - loss: 5.4726 - accuracy: 0.1718 - val_loss: 5.6061 - val_accuracy: 0.2583
Epoch 3/3
901/901 [==============================] - 3043s 3s/step - loss: 5.7464 - accuracy: 0.1741 - val_loss: 4.7445 - val_accuracy: 0.2583


This took us around 2 hours when connected with gpu.

and it seems that if you run it for 10 epochs you may get 92% accuracy or the validation set. So it certifies that the SOTA model works nicely.


If you love this project do leave a **star :)**

